<a href="https://colab.research.google.com/github/santiagosanchez15/Project1-comp215/blob/main/Comp_215_prject_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 comp 215

**Author:** Santiago Sanchez Covarrubias

**resources**: *Think python, Claude.ai*
  - https://allendowney.github.io/ThinkPython/. Think python URL


**Objectives**
- The creation of a SLM capable of to predict the third word

**Project description**

The project will develop a SLM capable of predicting the third word.

This project will be focus not only on developing the SLM but also on documenting the process.
Starting by adding different sections, that at the end of different sections will join all the pieces together.

After the SLM has been built with feeded data, the final SLM will be created by inhereting everything from the first one, the difference is tyhat this final version will not only take the feeded data through files but aslo through the Wikimedia REST API, the perfect source for thousand of wirtten texts.

At the end of all the documentation the full code will be available.





In [145]:
import string
from collections import Counter

# Parsed and clean function

  - get_text:
    - will get the text from the file

  - clean_text
    - will iterate through all the text, check if there are punctuations signs, remove and create a new list of words

  - bigrams -> words
    - check the anount aof times a word repeats after an specific word, then added to dictionary
      "Hello how are you"
      1 - (Hello how)
      2- (how are)
      3- (are you)
      amount of times the second word will come after the first one
      {"Hello", {"how": 2, "this": 4}} etc.


  - trigrams -> 3 words
    - same like bigrams but instead teh combination of 3 words
      next two words plus the word checking such as
      "The castle is big and made of stone"
      1 - (the castle is)
      2- (castle is big)
      3- (is big and)
      aount of times the next two words will come after the first one
      {'The': {"castle is", 3}
      {"The': {"red carpet", 2}
      etc, next two words following the first one there fore key can be tuple

In [127]:

def clean_text_from_file(file_name: str) -> list:
  '''from a given file returns a list of strings with the texted parsed and cleaned '''

  with open(file_name, 'r') as text: #open file given
    return [word.strip(string.punctuation).lower()  for line in text for word in line.split() if word.strip(string.punctuation)] #iterate through each word and strip to get clean word

assert clean_text_from_file('sample.txt')[:2] == ['hello', 'world']
assert clean_text_from_file('sample.txt')[-1] == 'wonderful'

In [152]:
def clean_text(string_text: str) -> list:
  '''Returns list of word cleaned '''
  return [word.strip(string.punctuation).lower() for word in string_text.split() if word.strip(string.punctuation)]

In [153]:
list1 = "Hello! my? friend is you!!!!!"
print(clean_text(list1))

['hello', 'my', 'friend', 'is', 'you']


#Get bigram
get the bigram from the given list return bigram

In [130]:
def get_bigrams(list_word: list) -> list:
  '''Returns a list of tuple of bigrams from a given list '''

  return list(zip(list_word[:-1], list_word[1:]))

In [154]:
list_word = ['hello', 'how', 'are', 'you', 'my', 'friend']
#expected output = (hello, how), (how, are)
for item in get_bigrams(list_word):
  print(item)

#Testing with previous function
assert get_bigrams(clean_text_from_file('sample.txt'))[:2] == [('hello', 'world'), ('world', 'how')]

new_bigram = get_bigrams(clean_text('hello my friend is you'))
print(new_bigram)

('hello', 'how')
('how', 'are')
('are', 'you')
('you', 'my')
('my', 'friend')
[('hello', 'my'), ('my', 'friend'), ('friend', 'is'), ('is', 'you')]


#Merge_dictionaries

function that will take two dictionaries and merge the two of them adding the elements and counting the bigrams

In [132]:
def merge_dictionaries(old_dict: dict, new_dict: dict) -> dict:
  '''Returns dictionary with updated values '''

  counter1, counter2 = 0,0

  all_keys = set(old_dict.keys() | new_dict.keys()) #we crate a set to get all the keys of both dicitionaries merging them
  result = {}

  for key in all_keys: #iterate thorugh they keys of both dictionaries
    counter1 = Counter(old_dict.get(key, {})) #use Counter function to get attributes
    counter2 = Counter(new_dict.get(key, {}))
    result[key] = dict(counter1 + counter2) #add the attributes to a new dictionary form given key

  return result



In [134]:
dict1 = {"hello": {"this": 1, "how": 2}, "apple": {"a": 1, "are": 2}}
dict2 = {"hello": {"how": 3}, "pear": {"yummy": 1}}
new_dict = merge_dictionaries(dict1, dict2)
print(new_dict)

{'apple': {'a': 1, 'are': 2}, 'hello': {'this': 1, 'how': 5}, 'pear': {'yummy': 1}}


# word_frequency
this function will take a dictionary and a given list of bigrams to update the dictionary given with the values corresponding to the frequency of the words appearance

In [135]:
def word_frequency(bigrams:list) -> dict:
  '''Returns a dictionary with updated frequency of words '''

  new_dict = {}
  for key, value in bigrams: #iterate trhough every element in the list of bigrams tuples
    if key not in new_dict: new_dict[key] = {} #create a new key if the key doenst exist
    if value not in new_dict[key]: new_dict[key][value] = 1 #give a value of 1 if the value doesnt exist
    else: new_dict[key][value] += 1 #update the value once the word is found


  return new_dict
#try function



In [136]:
#try function above

bigrams1  = get_bigrams(clean_text_from_file('sample.txt'))
new_dict = word_frequency(get_bigrams(clean_text_from_file('sample.txt')))
print(new_dict)

{'hello': {'world': 1}, 'world': {'how': 1}, 'how': {'are': 1}, 'are': {'you': 1}, 'you': {'today': 1}, 'today': {"i'm": 1}, "i'm": {'doing': 1}, 'doing': {'great': 1}, 'great': {"let's": 1}, "let's": {'explore': 1}, 'explore': {'coding': 1}, 'coding': {'writing': 1}, 'writing': {"creativity—it's": 1}, "creativity—it's": {'wonderful': 1}}


#word_frequency_from_file

Lets join all the functions together into a single function
it will take a file name as a paramter and return the dictionary that will be used to feed the model

In [159]:
def word_frequency_from_file(file_name: str, old_dict: dict) -> dict:
  '''Updates dictionary of frequencies from a given file '''

  text = clean_text_from_file(file_name) #get the clean text as a list
  bigrams = get_bigrams(text) #get bigrams form the zip function
  frequency = word_frequency(bigrams) #get a new dictioanry of frequencies
  return merge_dictionaries(old_dict, frequency) #returns the updated dictionary


In [166]:
#Test function
new_dict = {}
list_files = ['text1.txt', 'text2.txt']
for file in list_files:
  new_dict = word_frequency_from_file(file, new_dict)

print(new_dict)


{'cat': {'jumped': 2, 'played': 1}, 'more': {'than': 1}, 'the': {'big': 4, 'small': 2, 'red': 5, 'green': 3}, 'fast': {'the': 2, 'and': 1}, 'they': {'ran': 1}, 'red': {'apple': 3, 'car': 2}, 'my': {'favorites': 1}, 'car': {'was': 1, 'looked': 1, 'were': 1}, 'slow': {'but': 1}, 'but': {'the': 1}, 'apple': {'was': 1, 'tasted': 1, 'and': 1, 'more': 1}, 'jumped': {'high': 2, 'higher': 1}, 'i': {'like': 1}, 'ran': {'fast': 2, 'slow': 1}, 'was': {'sweet': 1, 'fast': 1}, 'looked': {'nice': 1}, 'good': {'and': 1}, 'dog': {'ran': 2, 'and': 1}, 'favorites': {'i': 1}, 'and': {'the': 3, 'jumped': 1}, 'than': {'the': 1}, 'high': {'the': 1}, 'nice': {'the': 1}, 'like': {'the': 1}, 'played': {'together': 1}, 'were': {'my': 1}, 'sweet': {'the': 1}, 'higher': {'the': 1}, 'green': {'apple': 2, 'car': 1}, 'small': {'dog': 1, 'cat': 1}, 'together': {'they': 1}, 'tasted': {'good': 1}, 'big': {'dog': 2, 'cat': 2}}


lets test the function with more complex files

In [167]:
#test function complex files
new_dict = {}
new_file_list = ['bigram_test1.txt', 'bigram_test2.txt']
for complex_file in new_file_list:
  new_dict = word_frequency_from_file(complex_file, new_dict)

new_dict

{'cat': {'jumped': 2, 'played': 1},
 'more': {'than': 1},
 'the': {'big': 4, 'small': 2, 'red': 5, 'green': 3},
 'fast': {'the': 2, 'and': 1},
 'they': {'ran': 1},
 'red': {'apple': 3, 'car': 2},
 'my': {'favorites': 1},
 'car': {'was': 1, 'looked': 1, 'were': 1},
 'slow': {'but': 1},
 'but': {'the': 1},
 'apple': {'was': 1, 'tasted': 1, 'and': 1, 'more': 1},
 'jumped': {'high': 2, 'higher': 1},
 'i': {'like': 1},
 'ran': {'fast': 2, 'slow': 1},
 'was': {'sweet': 1, 'fast': 1},
 'looked': {'nice': 1},
 'good': {'and': 1},
 'dog': {'ran': 2, 'and': 1},
 'favorites': {'i': 1},
 'and': {'the': 3, 'jumped': 1},
 'than': {'the': 1},
 'high': {'the': 1},
 'nice': {'the': 1},
 'like': {'the': 1},
 'played': {'together': 1},
 'were': {'my': 1},
 'sweet': {'the': 1},
 'higher': {'the': 1},
 'green': {'apple': 2, 'car': 1},
 'small': {'dog': 1, 'cat': 1},
 'together': {'they': 1},
 'tasted': {'good': 1},
 'big': {'dog': 2, 'cat': 2}}

#Word_frequency_from text
Ofcourse at this point of the project we can get the word frequency from file, bu twhat if we just want to copy and paste. well that is easy

In [163]:
def word_freqeuncy_from_text(given_string: str, old_dict: dict) -> str:
  '''Updates dictionary of frequencies from a given text '''

  new_list = clean_text(given_string)
  bigrams = get_bigrams(new_list) #get bigrams form the zip function
  frequency = word_frequency(bigrams) #get a new dictioanry of frequencies
  return merge_dictionaries(old_dict, frequency) #returns the updated dictionary

In [168]:
list1 = ['The big dog ran fast. The big cat jumped high. The small dog ran slow, but the small cat jumped higher. The big dog and the big cat played together. They ran fast and jumped high!', 'The red apple was sweet. The red car was fast. The green apple tasted good, and the green car looked nice. The red apple and the red car were my favorites. I like the red apple more than the green apple.']
dict1 = {}
for text in list1:
  dict1 = word_freqeuncy_from_text(text, dict1)
dict1

{'cat': {'jumped': 2, 'played': 1},
 'more': {'than': 1},
 'the': {'big': 4, 'small': 2, 'red': 5, 'green': 3},
 'fast': {'the': 2, 'and': 1},
 'they': {'ran': 1},
 'red': {'apple': 3, 'car': 2},
 'my': {'favorites': 1},
 'car': {'was': 1, 'looked': 1, 'were': 1},
 'slow': {'but': 1},
 'but': {'the': 1},
 'apple': {'was': 1, 'tasted': 1, 'and': 1, 'more': 1},
 'jumped': {'high': 2, 'higher': 1},
 'i': {'like': 1},
 'ran': {'fast': 2, 'slow': 1},
 'was': {'sweet': 1, 'fast': 1},
 'looked': {'nice': 1},
 'good': {'and': 1},
 'dog': {'ran': 2, 'and': 1},
 'favorites': {'i': 1},
 'and': {'the': 3, 'jumped': 1},
 'than': {'the': 1},
 'high': {'the': 1},
 'nice': {'the': 1},
 'like': {'the': 1},
 'played': {'together': 1},
 'were': {'my': 1},
 'sweet': {'the': 1},
 'higher': {'the': 1},
 'green': {'apple': 2, 'car': 1},
 'small': {'dog': 1, 'cat': 1},
 'together': {'they': 1},
 'tasted': {'good': 1},
 'big': {'dog': 2, 'cat': 2}}